# Example Solution: Predicting House Prices


## Loading the Data

In [ ]:
import findspark
findspark.init()
import pyspark

In [ ]:
data_dir = "../.assets/data/house/"

In [ ]:
!ls {data_dir}



After creating a `SparkSession`, we read the contents of the .csv file into a DataFrame. 

In [ ]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("HousePricePredictor") \
    .getOrCreate()


In [ ]:
data = spark.read \
    .format("csv") \
    .option("header", "true") \
    .load(f"{data_dir}/prices.csv") 


Defining a schema for this large dataframe beforehand is a daunting task, so we leave the types a the default (string) and cast later as needed. We know however that the prices should be floating point numbers:

In [ ]:
data = data.withColumn("SalePrice", data["SalePrice"].cast("DOUBLE"))

This DataFrame has a large number of columns - let's select some to take a brief look:

---------

## Example Solution 1: A Simple First Model

This is a minimalistic model to get us started. We use a bit of domain knowledge to select a couple of promising features that are already in a format which the ML algorithm can process.

In [ ]:
discrete_features = [
                "OverallQual",
                "OverallCond",
                "YearBuilt",
                ]

continuous_features = [
                "LotArea",
                "GrLivArea",
                "PoolArea"
                ]

In [ ]:
for col_name in discrete_features:
    data = data.withColumn(col_name, data[col_name].cast("INT"))
    
for col_name in continuous_features:
    data = data.withColumn(col_name, data[col_name].cast("DOUBLE"))

In [ ]:
feature_cols = discrete_features + continuous_features

In [ ]:
from pyspark.ml.feature import VectorAssembler

assemble_features = VectorAssembler(inputCols=feature_cols, 
                                    outputCol="features")

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

regression = DecisionTreeRegressor(featuresCol="features")

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assemble_features, regression])

-------

## Evaluation

Here we evaluate the performance of the regression model. A better model produces smaller errors in the predicted price. The two error metrics we use are **Root-Mean-Squared-Error (RMSE)** and **Mean Average Error (MAE)** between the predicted value and the observed sales price. In order to get robust scores with less random fluctuation, we apply **cross-validation**.

In [ ]:
import pandas
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
rmse = CrossValidator(estimator=pipeline,
                    evaluator=RegressionEvaluator(metricName="rmse", labelCol="label", predictionCol="prediction"),
                    estimatorParamMaps=ParamGridBuilder().build(),
                    numFolds=4) \
                    .fit(data.withColumnRenamed("SalePrice", "label")) \
                    .avgMetrics[0]

mae = CrossValidator(estimator=pipeline,
                    evaluator=RegressionEvaluator(metricName="mae", labelCol="label", predictionCol="prediction"),
                    estimatorParamMaps=ParamGridBuilder().build(),
                    numFolds=4) \
                    .fit(data.withColumnRenamed("SalePrice", "label")) \
                    .avgMetrics[0]

### Result

In [ ]:
import datetime
team_name = "Minimalists"
print("\t".join(["time", "team", "RMSE", "MAE"]))
print("\t".join([datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), team_name, "{0:.4f}".format(rmse), "{0:.4f}".format(mae)]))

## Diagnostics

In order to get a better understanding of the error made by the model, plot the distribution of prices, predicted prices, and errors. This can provide useful feedback for model improvement.

In [ ]:
predicted = pipeline.fit(data.withColumnRenamed("SalePrice", "label")).transform(data)
predicted[["SalePrice", "prediction"]].show()

In [ ]:
predicted_pd = predicted[["SalePrice", "prediction"]].toPandas()

In [ ]:
import seaborn
seaborn.set_style("whitegrid")

In [ ]:
seaborn.histplot(predicted_pd["SalePrice"], kde=True)


In [ ]:
seaborn.histplot(predicted_pd["prediction"], kde=True)


In [ ]:
seaborn.histplot(predicted_pd["SalePrice"] - predicted_pd["prediction"], kde=True)


------------

## Example Solution 2: Philipp's Model

Data scientist Philipp also gave house price prediction a try. Experienced in machine learning and predictive modeling, but new to Spark, he spent a few hours and built a more elaborate model. He also described his workflow for us. Let's follow the steps he took - particularly his feature engineering - and see how his model is doing compared to the minimalistic one.

In [ ]:
from pyspark.ml import Transformer, Estimator, Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer

> First I take a look at all the columns and decide which one I want to use at first. I choose numeric attributes like number of square feet of different parts of the house or the number of bedrooms, but also "string" attributes like the quality description of different parts of the house. In this case, the latter are quite easy to translate to numeric attributes the machine will understand (f.e. for Basement Quality "Ex" (Excellent) is a 1, "Po" (Poor) is a 6). I even take one categorical attribute into account, the "SaleCondition". Because there are only a handful of options for this attribute, I will choose One-Hot-Encoding to prepare it for machine learning. I also split the features into numeric features, features which are some kind of rating and easily convertible to numbers, and the categorical feature. This way it should be easy to add more features later on if needed.

In [ ]:
numeric_features = ['OverallQual', 'OverallCond', 'YearBuilt', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 
                    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
                    'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 
                    'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
                    'PoolArea']
rating_features = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                   'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish',
                   'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']
categorical_features = ['SaleCondition']

features = numeric_features+rating_features+categorical_features

target = ['SalePrice']

> Unfortunately, we cannot simply use show to display the features in an easy way. We have to split the visualization.

In [ ]:
data[features[:10]].show()

In [ ]:
data[features[10:20]].show()

In [ ]:
data[features[20:30]].show()

In [ ]:
data[features[30:40]].show()

In [ ]:
data[features[40:]].show()

> So let's build a simple transformer which shortens our data to only the features we specified here.

In [ ]:
class ColumnDropper(Transformer):
    """
    Reduce data to only a subset of columns
    """
    
    # lazy workaround - a transformer needs to have these attributes
    # TODO: replace if needed
    _defaultParamMap = dict()
    _paramMap = dict()
    _params = dict()
    uid = 0

    def __init__(self, cols=None):
        self.cols = cols


    def _transform(self, data):
        dataAfterDrop = data[self.cols] 
        return dataAfterDrop
    
    def __repr__(self):
        """ Show a proper string representation when printing the pipeline stage"""
        return str(type(self))

In [ ]:
select_pipeline = Pipeline(stages=[ColumnDropper(cols=features+target)])
data_short = select_pipeline.fit(data).transform(data)

**Looking for Missing Values**

> Let's use the code from the previous example to check for missing values.

In [ ]:
for col in data_short.columns:
    print(col, " : ", data_short.filter(f"{col} is NULL").count())

> No missing values!

**Converting the attribute data types**

> Now I can start converting the data types of the attributes. For numeric features this is simple, it gets hard however when I try convert a rating-type attribute to numeric values. Unfortunately, the data does not follow a uniform convention on how to rate things. For every type of rating, I have to build a translation table... This could be automated using the documentation file, but in this case (because I do not expect to do this again) I choose the manual approach. I create a table index to specify which column uses which table, and then I create the translation tables itself, where every string value is given a numeric value. 

This is a nice example for how much work could be saved by already thinking about later uses of the data when collecting it - a uniform rating scheme would make things simpler. Well, as data scientist we are used to spending most of our time cleaning data, so let's get to it.

In [ ]:
table_index = {'ExterQual' : 0,
               'ExterCond' : 0,
               'BsmtQual' : 0,
               'BsmtCond' : 0,
               'BsmtExposure' : 1,
               'BsmtFinType1' : 2,
               'HeatingQC' : 0,
               'CentralAir' : 3,
               'KitchenQual' : 0,
               'Functional' : 4,
               'FireplaceQu' : 0,
               'GarageFinish' : 5,
               'GarageQual' : 0,
               'GarageCond' : 0,
               'PavedDrive' : 6,
               'PoolQC' : 7,
               'Fence' : 8}


translation_tables = []
translation_tables.append({'Ex' : '0', # I have to use strings, because df.na.replace does
                       'Gd' : '1', # not support mixed type replacing
                       'TA' : '2',
                       'Fa' : '3',
                       'Po' : '4',
                       'NA' : '5'})

translation_tables.append({'Gd' : '0',
                       'Av' : '1',
                       'Mn' : '2',
                       'No' : '3',
                       'NA' : '4'})

translation_tables.append({'GLQ' : '0',
                       'ALQ' : '1',
                       'BLQ' : '2',
                       'Rec' : '3',
                       'LwQ' : '4',
                       'Unf' : '5',
                       'NA' : '6'})

translation_tables.append({'N' : '0',
                       'Y' : '1'})

translation_tables.append({'Typ' : '0',
                       'Min1' : '1',
                       'Min2' : '2',
                       'Mod' : '3',
                       'Maj1' : '4',
                       'Maj2' : '5',
                       'Sev' : '6',
                       'Sal' : '7'})

translation_tables.append({'Fin' : '0',
                       'RFn' : '1',
                       'Unf' : '2',
                       'NA' : '3'})

translation_tables.append({'Y' : '0',
                       'P' : '1',
                       'N' : '2'})

translation_tables.append({'Ex' : '0',
                       'Gd' : '1',
                       'TA' : '2',
                       'Fa' : '3',
                       'NA' : '4'})

translation_tables.append({'GdPrv' : '0',
                       'MnPrv' : '1',
                       'GdWo' : '2',
                       'MnWw' : '3',
                       'NA' : '4'})


In [ ]:
class TypeConverter(Transformer):
    """
    Converts set of columns to numeric types
    """
    
    # lazy workaround - a transformer needs to have these attributes
    # TODO: replace if needed
    _defaultParamMap = dict()
    _paramMap = dict()
    _params = dict()
    uid = 0

    def __init__(self, numeric_features=None, rating_features=None, table_index=None, translation_tables=None):
        self.numeric_features = numeric_features # List of all numeric columns
        self.rating_features = rating_features # List of all rating columns
        self.table_index = table_index # What translation table to use for what rating column
        self.translation_tables = translation_tables


    def _transform(self, data):
        for col in data.columns:
            if col in self.numeric_features:
                data = (data.withColumn(col, data[col].cast("double")))
            elif col in self.rating_features:
                trans_table_nr = self.table_index[col]
                data = data.na.replace(self.translation_tables[trans_table_nr], value=None, subset=col)
                data = (data.withColumn(col, data[col].cast("double")))
            #else:
                #print("Did not convert column " + col + " with transformer TypeConverter.")
        return data
    
    def __repr__(self):
        """ Show a proper string representation when printing the pipeline stage"""
        return str(type(self))

> All we need now is to convert the attribute SaleCondition with OneHotEncoding. We also drop the temporarily created column SaleCondition_index and the column SaleCondition and proceed only with the One-Hot encoded variant.

In [ ]:
stages = []
stages.append(StringIndexer(inputCol='SaleCondition',
                            outputCol='SaleCondition_index')) # One-Hot Encoding does not work with strings...
stages.append(OneHotEncoderEstimator(inputCols=['SaleCondition_index'],
                                     outputCols=['SaleCondition_onehot']))
stages.append(TypeConverter(numeric_features=numeric_features+target, rating_features=rating_features, 
                                                  table_index=table_index, translation_tables=translation_tables))
features_new = features.copy()
features_new.remove('SaleCondition')
features_new.append('SaleCondition_onehot')
stages.append(ColumnDropper(cols=features_new+target))
preproc_pipeline = Pipeline(stages=stages)


In [ ]:
%time data_preproc = preproc_pipeline.fit(data_short).transform(data_short)

** Assembling all feature columns into one single columns and renaming the target column to label** 



In [ ]:
class LabelRenamer(Transformer):
    """
    Drops rows with at least one not-a-number element
    """
    
    # lazy workaround - a transformer needs to have these attributes
    # TODO: replace if needed
    _defaultParamMap = dict()
    _paramMap = dict()
    _params = dict()
    uid = 0

    def __init__(self, col=None):
        self.col = col


    def _transform(self, data):
        dataRenamed = data.withColumnRenamed(self.col, 'label')
        return dataRenamed
    
    def __repr__(self):
        """ Show a proper string representation when printing the pipeline stage"""
        return str(type(self))

In [ ]:
from pyspark.ml.feature import VectorAssembler

stages = []
stages.append(LabelRenamer(col=target[0]))
stages.append(VectorAssembler(inputCols=features_new, outputCol='features'))
spark_needs_this_pipeline = Pipeline(stages=stages)
data_sparked = spark_needs_this_pipeline.fit(data_preproc).transform(data_preproc)

**Using regression to predict the target variable**

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

model_pipeline = Pipeline(stages=[RandomForestRegressor(featuresCol = 'features', labelCol = 'label')])
data_model = model_pipeline.fit(data_sparked).transform(data_sparked)

**Joining all pipelines to a single pipeline**

In [ ]:
pipeline = Pipeline(stages=[select_pipeline, preproc_pipeline, spark_needs_this_pipeline, model_pipeline])

-----

How does Philipp's model perform? We run our evaluation code. Compare the results to the minimal model above.

## Evaluation

Here we evaluate the performance of the regression model. A better model produces smaller errors in the predicted price. The two error metrics we use are **Root-Mean-Squared-Error (RMSE)** and **Mean Average Error (MAE)** between the predicted value and the observed sales price. In order to get robust scores with less random fluctuation, we apply **cross-validation**.

In [ ]:
import pandas
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
rmse = CrossValidator(estimator=pipeline,
                    evaluator=RegressionEvaluator(metricName="rmse", labelCol="label", predictionCol="prediction"),
                    estimatorParamMaps=ParamGridBuilder().build(),
                    numFolds=4) \
                    .fit(data) \
                    .avgMetrics[0]

mae = CrossValidator(estimator=pipeline,
                    evaluator=RegressionEvaluator(metricName="mae", labelCol="label", predictionCol="prediction"),
                    estimatorParamMaps=ParamGridBuilder().build(),
                    numFolds=4) \
                    .fit(data) \
                    .avgMetrics[0]

### Result

In [ ]:
import datetime
team_name = "Philipp"
print("\t".join(["time", "team", "RMSE", "MAE"]))
print("\t".join([datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), team_name, "{0:.4f}".format(rmse), "{0:.4f}".format(mae)]))

## Diagnostics

In order to get a better understanding of the error made by the model, plot the distribution of prices, predicted prices, and errors. This can provide useful feedback for model improvement.

In [ ]:
predicted = pipeline.fit(data).transform(data)
predicted[["label", "prediction"]].show()

In [ ]:
predicted_pd = predicted[["label", "prediction"]].toPandas()

In [ ]:
import seaborn
seaborn.set_style("whitegrid")

In [ ]:
seaborn.histplot(predicted_pd["label"], kde=True)


In [ ]:
seaborn.histplot(predicted_pd["prediction"], kde=True)


In [ ]:
seaborn.histplot(predicted_pd["label"] - predicted_pd["prediction"], kde=True)


---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2025 [Point 8 GmbH](https://point-8.de)_